In [3]:
from langchain.llms import GooglePalm
api_key="Your_GooglePaLM_API_key"
llm=GooglePalm(google_api_key=api_key,temperature=0.2)

In [4]:
test=llm("how is the weather")

In [5]:
print(test)

35 °F, partly cloudy


In [6]:
from langchain.document_loaders import CSVLoader
loader=CSVLoader(file_path='FAQ.csv',source_column='Prompt')
data = loader.load()

In [7]:
data

[Document(page_content='Prompt: How do I exchange my existing commercial vehicle?\nanswer: To exchange your commercial vehicle click on the Exchange button in the Navigational Menu Bar. \r\nClicking on the exchange button leads to the TATA OK page which helps in buying, \r\nselling or exchanging commercial vehicles of any make, model or vintage.', metadata={'source': 'How do I exchange my existing commercial vehicle?', 'row': 0}),
 Document(page_content='Prompt: How do I get to know the final offer price after I have booked my vehicle?\nanswer: The booking amount paid by you Is used to simply reserve the vehicle in your name. The final offer price is communicated by the dealer.', metadata={'source': 'How do I get to know the final offer price after I have booked my vehicle?', 'row': 1}),
 Document(page_content='Prompt: How do I know my booking has been confirmed?\nanswer: Once you have completed the payment, you will receive a Thank You message confirming your payment. \r\nThis denotes

In [8]:
# %pip install langchain sentence_transformers
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()

In [9]:
testembed=embeddings.embed_query('who are you')


In [10]:
len(testembed)

768

In [11]:
type(testembed)

list

In [12]:
print(testembed[:5])

[0.07499957084655762, 0.04322311282157898, -0.0005882302648387849, 0.005776368547230959, 0.047345250844955444]


In [13]:
# %pip install faiss-cpu
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings()
db = await FAISS.afrom_documents(data, embeddings)



In [14]:
retriever =db.as_retriever()
rdata = retriever.get_relevant_documents ("I want to exchange my commercial vehicle")
o=rdata

In [15]:
print(o)

[Document(page_content='Prompt: How do I exchange my existing commercial vehicle?\nanswer: To exchange your commercial vehicle click on the Exchange button in the Navigational Menu Bar. \r\nClicking on the exchange button leads to the TATA OK page which helps in buying, \r\nselling or exchanging commercial vehicles of any make, model or vintage.', metadata={'source': 'How do I exchange my existing commercial vehicle?', 'row': 0}), Document(page_content='Prompt: How do I calculate the EMI for the commercial vehicle I want to buy?\nanswer: Once you have opened a vehicle page, you can scroll below the ‘Specifications’ section and the Video section to view the EMI Calculator section. \r\nUse the sliders present for various parameters such as ‘Loan amount’, ‘Rate of Interest’ and ‘Loan Tenure’ to calculate the \r\nEMI you will need to pay for your preferred commercial vehicle.', metadata={'source': 'How do I calculate the EMI for the commercial vehicle I want to buy?', 'row': 4}), Document(

In [16]:
from langchain.prompts import PromptTemplate

prompt_temp="""Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
if the answer is not found in the context, kindly state "This Question not Present in My Database." Don't try to make up an answer.
CONTEXT: {context}
QUESTION: {question}"""
PROMPT = PromptTemplate(template=prompt_temp, input_variables=["context", "question"])


In [19]:
from langchain.chains import RetrievalQA
chain = RetrievalQA.from_chain_type(llm=llm,chain_type="stuff", retriever=retriever, input_key="query", return_source_documents=False,chain_type_kwargs={"prompt" : PROMPT})

In [20]:
chain("when the sale was high and down")

{'query': 'when the sale was high and down',
 'result': 'The sales were high in 2019 and down in 2020.'}

In [148]:
chain('give the profit of this compamy')

{'query': 'give the profit of this compamy',
 'result': '30 thousand core',
 'source_documents': [Document(page_content='Prompt: what is the profit in 2020\nanswer: 30 thousand core', metadata={'source': 'what is the profit in 2020', 'row': 10}),
  Document(page_content='Prompt: how much of workers works in the company\nanswer: around 20000 people', metadata={'source': 'how much of workers works in the company', 'row': 11}),
  Document(page_content='Prompt: What is the maximum price of the car\nanswer: one core', metadata={'source': 'What is the maximum price of the car', 'row': 5}),
  Document(page_content='Prompt: when the sales was down\nanswer: 2020', metadata={'source': 'when the sales was down', 'row': 9})]}

In [149]:
chain('which year they got this salray')

{'query': 'which year they got this salray',
 'result': 'This Question not Present in My Database.',
 'source_documents': [Document(page_content='Prompt: when the sales was down\nanswer: 2020', metadata={'source': 'when the sales was down', 'row': 9}),
  Document(page_content='Prompt: which year the sell of cars were high\nanswer: 2019', metadata={'source': 'which year the sell of cars were high', 'row': 8}),
  Document(page_content='Prompt: what is the profit in 2020\nanswer: 30 thousand core', metadata={'source': 'what is the profit in 2020', 'row': 10}),
  Document(page_content='Prompt: What is the minimum price of the car\nanswer: ten lakh', metadata={'source': 'What is the minimum price of the car', 'row': 6})]}

In [150]:
chain('what is salary of this company in 2018')

{'query': 'what is salary of this company in 2018',
 'result': ' This Question not Present in My Database.',
 'source_documents': [Document(page_content='Prompt: how much of workers works in the company\nanswer: around 20000 people', metadata={'source': 'how much of workers works in the company', 'row': 11}),
  Document(page_content='Prompt: what is the profit in 2020\nanswer: 30 thousand core', metadata={'source': 'what is the profit in 2020', 'row': 10}),
  Document(page_content='Prompt: which year the sell of cars were high\nanswer: 2019', metadata={'source': 'which year the sell of cars were high', 'row': 8}),
  Document(page_content='Prompt: What is the minimum price of the car\nanswer: ten lakh', metadata={'source': 'What is the minimum price of the car', 'row': 6})]}

In [155]:
chain('what is the profit of this company in 2021?')

{'query': 'what is the profit of this company in 2021?',
 'result': 'This Question not Present in My Database.',
 'source_documents': [Document(page_content='Prompt: what is the profit in 2020\nanswer: 30 thousand core', metadata={'source': 'what is the profit in 2020', 'row': 10}),
  Document(page_content='Prompt: when the sales was down\nanswer: 2020', metadata={'source': 'when the sales was down', 'row': 9}),
  Document(page_content='Prompt: how much of workers works in the company\nanswer: around 20000 people', metadata={'source': 'how much of workers works in the company', 'row': 11}),
  Document(page_content='Prompt: which year the sell of cars were high\nanswer: 2019', metadata={'source': 'which year the sell of cars were high', 'row': 8})]}